In [ ]:
import uproot
import glob
import numpy as np
import pandas as pd
import time
import os
import re

import pyvista as pv
pv.set_jupyter_backend('trame')  # or 'panel' if using panel

from scipy.constants import epsilon_0, e as q_e
from scipy.interpolate import griddata
from scipy.optimize import curve_fit
from scipy.spatial import cKDTree
from scipy.interpolate import interp1d

# Import interpolate for numerical method
from scipy.interpolate import CubicSpline
import matplotlib.gridspec as gridspec

from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib as mpl
import matplotlib.ticker as ticker

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed

import trimesh
import h5py
from trimesh.points import PointCloud

from common_functions import *  # Assuming common_functions.py is in the same directory


In [ ]:
# read in geometry to get the offset 
geometryIN = "stacked_spheres_frompython_cropped.stl"
geometry = trimesh.load_mesh(f'geometry/{geometryIN}') 

## 2D Representation of the Electric Field

In [ ]:
iteration = 99

configIN = "onlyphotoemission"
#directory = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/build-dissipationRefinedGrid-initial8max0.8final12/"
description = "ZimmermanInitial8max0.8final11"

if iteration <10 :
    filenames = sorted(glob.glob(f"raw-files/{description}/*00{iteration}*{configIN}*.txt")) #{iteration}
else:
    filenames = sorted(glob.glob(f"raw-files/{description}/*{iteration}*{configIN}*.txt")) #{iteration}
print(filenames)

df  = read_data_format_efficient(filenames,scaling=True)

# check to make sure this matches the total nodes in outputlogs
#len(df[iteration]["E_mag"]) 

In [ ]:
## SETTINGS HERE ARE OPTIMIZED FOR ITERATION 86 ##

fieldIN = df[iteration]

N_DOWNSAMPLE_EMAG = 1
ARROW_VOXEL_SPACING = 0.02 
Y_SLICE = 0.0 + geometry.centroid[1]  # This correctly uses the centroid's Y
THICKNESS = 0.001
VECTOR_SCALE_FACTOR = 1.5e-7
FIELD_AVERAGE_RADIUS = 2.5e-3

vmin, vmax = (-2e5, 2e5)
red_point = np.array([-0.1, 0, 0.1 + 0.037]) + geometry.centroid

# ----------------------------------------------------
# Voxel Downsampling Helper Function
# ----------------------------------------------------
def voxel_downsample_points(points, spacing):
    """
    Selects one point per voxel defined by the spacing.
    Assumes points are 3D, but only uses X and Z for 2D density control.
    """
    min_x, _, min_z = points.min(axis=0)
    
    x_indices = np.floor((points[:, 0] - min_x) / spacing).astype(int)
    z_indices = np.floor((points[:, 2] - min_z) / spacing).astype(int)
    
    max_x_index = x_indices.max() + 1
    voxel_keys = z_indices * max_x_index + x_indices

    _, unique_indices = np.unique(voxel_keys, return_index=True)
    
    return unique_indices

# ----------------------------------------------------
# Step 0: Load, Filter, and Downsample Data
# ----------------------------------------------------
start_time = time.time()
points = fieldIN["pos"]
vectors = fieldIN["E"]
magnitudes = fieldIN["E_mag"]

initial_mask = (points[:, 2] > 0) & (magnitudes > 0)
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]

points_ds = points[::N_DOWNSAMPLE_EMAG]
vectors_ds = vectors[::N_DOWNSAMPLE_EMAG]
magnitudes_ds = magnitudes[::N_DOWNSAMPLE_EMAG]

point_cloud = pv.PolyData(points_ds)
point_cloud["E_mag"] = magnitudes_ds
point_cloud["Ex_val"] = vectors_ds[:,0]
point_cloud["Ez_val"] = vectors_ds[:,2]

print(f"Starting points (filtered by z > 0 & mag > 0): {len(points)}")

# ----------------------------------------------------
# Step 1: Geometry Setup and Slicing
# ----------------------------------------------------
start_time_geo = time.time()

pv_spheres = pv.PolyData(
    geometry.vertices,
    np.hstack([np.full((len(geometry.faces), 1), 3), geometry.faces])
).compute_normals()

bbox_bounds = point_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = pv_spheres.clip_box(bbox, invert=False)

normal = [0, 1, 0]

# FIX: Use explicit center with Y_SLICE for the plane
plane_center = [
    (bbox_bounds[0] + bbox_bounds[1]) / 2,  # X center
    Y_SLICE,                                  # Y at slice location
    (bbox_bounds[4] + bbox_bounds[5]) / 2   # Z center
]

field_slice_mesh = pv.Plane(
    center=plane_center,  # <-- FIXED: Use explicit center at Y_SLICE
    direction=normal,
    j_size=bbox_bounds[1] - bbox_bounds[0],
    i_size=bbox_bounds[5] - bbox_bounds[4],
    i_resolution=250, 
    j_resolution=250
)

field_slice_interpolated = field_slice_mesh.interpolate(
    point_cloud,
    sharpness=3.0,
    radius=0.001,
    null_value=1, 
    strategy='closest_point'
)

# Ensure all Y coordinates are exactly at Y_SLICE
field_slice_interpolated.points[:, 1] = Y_SLICE

# Slice geometry at the same location
geo_slice = pv_spheres_cropped.slice(normal=normal, origin=plane_center)  # <-- FIXED: Use plane_center
print(f"Geometry and slicing preparation complete in {time.time() - start_time_geo:.2f}s")

# ----------------------------------------------------
# Step 2: Vector Field Glyphs (Arrows)
# ----------------------------------------------------
start_time_vectors = time.time()

vector_mask = np.abs(points_ds[:, 1] - Y_SLICE) < THICKNESS
points_slice_full = points_ds[vector_mask]
vectors_slice_full = vectors_ds[vector_mask]
magnitudes_slice_full = magnitudes_ds[vector_mask]

unique_indices = voxel_downsample_points(points_slice_full, ARROW_VOXEL_SPACING)

points_slice = points_slice_full[unique_indices]
vectors_slice = vectors_slice_full[unique_indices]
magnitudes_slice = magnitudes_slice_full[unique_indices]

# ----------------------------------------------------
# ADD ONE MORE COLUMN ON THE RIGHT EDGE
# ----------------------------------------------------
# Find the maximum X value in the current slice
max_x = points_slice[:, 0].max()

# Find all Z values that exist at or near the max X
x_threshold = max_x - ARROW_VOXEL_SPACING / 2  # Points in the rightmost column
rightmost_mask = points_slice[:, 0] >= x_threshold

# Get unique Z values from the rightmost column
z_values_right = points_slice[rightmost_mask, 2]

# Create new points one spacing to the right
new_x = max_x + ARROW_VOXEL_SPACING
new_points = np.array([[new_x, Y_SLICE - 2*THICKNESS, z] for z in z_values_right])

# Interpolate field values at these new points from the nearest neighbors
# Use the existing points to find nearest field values
from scipy.spatial import cKDTree
tree = cKDTree(points_slice_full[:, [0, 2]])  # Only use X and Z for 2D lookup
distances, indices = tree.query(new_points[:, [0, 2]], k=1)
new_vectors = vectors_slice_full[indices]
new_magnitudes = magnitudes_slice_full[indices]

# Append the new column to existing data
points_slice = np.vstack([points_slice, new_points])
vectors_slice = np.vstack([vectors_slice, new_vectors])
magnitudes_slice = np.concatenate([magnitudes_slice, new_magnitudes])
# ----------------------------------------------------

MAGNITUDE_MAX_CLAMP = ARROW_VOXEL_SPACING / VECTOR_SCALE_FACTOR / 2
magnitudes_slice_clamped = np.clip(magnitudes_slice, a_min=None, a_max=MAGNITUDE_MAX_CLAMP)

points_slice[:,1] = Y_SLICE - 2*THICKNESS
vectors_slice[:,1] = 0.0 - 2*THICKNESS
slice_mesh_vectors = pv.PolyData(points_slice)
slice_mesh_vectors['vectors'] = vectors_slice
slice_mesh_vectors['magnitude'] = magnitudes_slice_clamped

print(f"Points in vector slice (after density control + extra column): {len(points_slice)}, old length: {len(points_slice_full)}...")

arrow = pv.Arrow(tip_length=0.3, tip_radius=0.2, shaft_radius=0.04)
glyphs = slice_mesh_vectors.glyph(
    orient='vectors',
    scale='magnitude',
    factor=VECTOR_SCALE_FACTOR,
    geom=arrow
)

# ----------------------------------------------------
# Step 3: Visualization
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

pl.add_mesh(
    field_slice_interpolated,
    scalars="Ex_val",
    cmap="YlGnBu",
    opacity=1,
    show_edges=False,
    clim=[vmin, vmax],
    scalar_bar_args={
        'title': None,
        'vertical': False,
        'position_x': 0.20,
        'position_y': 0.12,
        'width': 0.6,
        'height': 0.05,
    }
)

pl.add_mesh(geo_slice, color="black", line_width=3, opacity=0.5)
pl.add_mesh(glyphs, color='black', show_scalar_bar=False, line_width=4, opacity=1)

sphere = pv.Sphere(radius=FIELD_AVERAGE_RADIUS, center=red_point)
pl.add_mesh(sphere, color="red", opacity=1)

pl.enable_parallel_projection()
pl.enable_2d_style()
pl.view_xz()

pl.screenshot(f'figures/fieldvectors_{configIN}#{iteration}.jpeg', scale=4)

print(f"Total execution time: {time.time() - start_time:.2f}s")
pl.show(jupyter_backend='static')

In [ ]:
# ----------------------------------------------------
# Step 3: Visualization Setup
# ----------------------------------------------------

# Define visualization parameters
FIELD_AVERAGE_RADIUS = 1e-3
CIRCLE_RADIUS = 40/2/1000  # 20 µm converted to meter

offsetLimitsX = 0.007
offsetLimitsY = 0.0092
new_step_x = offsetLimitsX   # 2 points in X
new_step_y = offsetLimitsY   # 3 points in Y

# Create grid of target points
xoffset = np.round(np.arange(-offsetLimitsX, offsetLimitsX + 1e-9, new_step_x), 5)[0:2] #[1]
yoffset = np.round(np.arange(-offsetLimitsY, offsetLimitsY + 1e-9, new_step_y), 5)
X, Y = np.meshgrid(xoffset, yoffset)

target_center = np.array([-0.1, 0, 0.1 + 0.037]) + geometry.centroid
target_points_array = np.vstack([
    target_center[0] - X.flatten(), 
    target_center[1] - np.zeros(len(X.flatten())), 
    target_center[2] - Y.flatten()
]).T

print(f"Processing {len(target_points_array)} target points with radius {FIELD_AVERAGE_RADIUS*1000:.1f} mm")

# ----------------------------------------------------
# Create Plotter and Add Meshes
# ----------------------------------------------------
pl = pv.Plotter()
pl.set_background('white')

# Add interpolated field slice
pl.add_mesh(
    field_slice_interpolated,
    scalars="Ex_val",
    cmap="YlGnBu",
    opacity=1,
    show_edges=False,
    clim=[vmin, vmax],
    scalar_bar_args={
        'title': None,
        'vertical': False,
        'position_x': 0.20,
        'position_y': 0.12,
        'width': 0.6,
        'height': 0.05,
    }
)

# Add geometry outline
pl.add_mesh(geo_slice, color="black", line_width=5, opacity=0.5)

# Add vector field arrows
pl.add_mesh(glyphs, color='black', show_scalar_bar=False, line_width=4, opacity=1)

# Color Map Setup
CMAP_NAME = 'jet'
n_targets = len(target_points_array)
discrete_cmap = plt.get_cmap(CMAP_NAME, n_targets + 1)
color_list_rgba = [discrete_cmap(i) for i in np.linspace(0, 1, n_targets + 1)]

# Add target point spheres
for target_point, colorIN in zip(target_points_array,color_list_rgba):
    sphere = pv.Sphere(radius=FIELD_AVERAGE_RADIUS, center=target_point)
    pl.add_mesh(sphere, color=colorIN, opacity=1)

# # Add reference circle at center (in XZ plane)
theta = np.linspace(0, 2 * np.pi, 100)
circle_points = np.column_stack([
    target_center[0] + CIRCLE_RADIUS * np.cos(theta),
    np.full_like(theta, target_center[1]),
    target_center[2] + CIRCLE_RADIUS * np.sin(theta)
])
polyline = pv.PolyData(circle_points)
pl.add_mesh(polyline, color='k', point_size=0.5, opacity=0.8) # Add to the plot

# ----------------------------------------------------
# Camera Setup and Render
# ----------------------------------------------------
pl.enable_parallel_projection()
pl.enable_2d_style()
pl.view_xz()

print(f"Total execution time: {time.time() - start_time:.2f}s")
pl.show(jupyter_backend='static')

In [ ]:
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 16})

cmap = plt.cm.YlGnBu 
vmin, vmax = (-2e5, 2e5) # in log(E_mag) units
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

fig, ax = plt.subplots(figsize=(10, 0.1))

# --- 1. Create and Configure the ScalarFormatter ---
formatter = ticker.ScalarFormatter(useMathText=True)

formatter.set_useOffset(False) 
formatter.set_powerlimits((0, 0)) 

# --- 2. Create the Colorbar and apply the Formatter ---
cb = mpl.colorbar.ColorbarBase(
    ax, 
    cmap=cmap, 
    norm=norm, 
    orientation='horizontal', label=r"E$_x$ (V/m)"
)

# Apply the formatter to the colorbar's x-axis
cb.ax.xaxis.set_major_formatter(formatter)

# --- 3. Display the Plot ---
plt.show()

## Quantitative Comparison with Zimmerman et al. 2016

In [ ]:
# test one file 

# directory = "processed-fieldmaps/"
# processedResults = load_h5_to_dict(f"{directory}/PE_425K_initial8max0.8final11_RefinedGridDissipation_sphere40um_adjustedworld_through2X.h5")

In [ ]:
# --- configuration ---
directory = "processed-fieldmaps/"
h5_filenames = glob.glob(f"{directory}/*425K*.h5")

FIELD_AVERAGE_RADIUS = 1e-3
CIRCLE_RADIUS = 40/2/1000  # 20 µm converted to meter

offsetLimitsX = 0.007
offsetLimitsY = 0.0092
new_step_x = offsetLimitsX   # 2 points in X
new_step_y = offsetLimitsY   # 3 points in Y

# Create grid of target points
xoffset = np.round(np.arange(-offsetLimitsX, offsetLimitsX + 1e-9, new_step_x), 5)[1] #[0:2] #[1]
yoffset = np.round(np.arange(-offsetLimitsY, offsetLimitsY + 1e-9, new_step_y), 5)[1] 

X, Y = np.meshgrid(xoffset, yoffset)

target_center =  np.array([-0.1, 0, 0.1 + 0.037]) + geometry.centroid
target_points_array = np.vstack([
    target_center[0] - X.flatten(), 
    target_center[1] - np.zeros(len(X.flatten())), 
    target_center[2] - Y.flatten()
]).T 

if len(target_points_array) == 1: print(f"Processing target point at {target_points_array} with radius {FIELD_AVERAGE_RADIUS*1000} um")
else: print(f"Processing {len(target_points_array)} target points with radius {FIELD_AVERAGE_RADIUS*1000} um")

# --- helper function for one key and one target point ---
def process_key_target(keyIN, val, target_point, radius):
    points = val["pos"]
    vectors = val["E"]
    magnitudes = val["E_mag"]
    
    # Mask for spherical averaging around target point
    mask = np.sum((points - target_point)**2, axis=1) <= radius**2
    
    if not np.any(mask):
        # Return placeholders if no points in sphere
        return -1, np.full(3, np.nan), np.nan, np.full(3, np.nan), 0

    avg_position = points[mask].mean(axis=0)
    E_vec = vectors[mask].mean(axis=0)
    E_mag = magnitudes[mask].mean(axis=0)
    E_vec_errors = vectors[mask].std(axis=0) / np.sqrt(len(magnitudes[mask]))
    
    return int(keyIN.split("_")[1]), E_vec, E_mag, avg_position, len(magnitudes[mask])

# --- extract metadata from filename ---
def parse_filename_metadata(filename):
    """
    Processes files and extracts metadata.
    Example:
    PE_425K_initial8max0.8final12_RefinedGridDissipation_500000particles_Sphere20um_pos-0.1_through26.h5
    """

    base = os.path.basename(filename)
    case = base.split("_")[0]

    # Temperature
    temp_match = re.search(r'_(\d+)K_', base)
    temperature = int(temp_match.group(1)) if temp_match else np.nan

    # Position
    pos_match = re.search(r'_pos([-+]?\d*\.?\d+)_through', base)
    pos_value = float(pos_match.group(1)) if pos_match else np.nan

    # Sphere size (if any)
    sphere_match = re.search(r'_sphere(\d+)um', base, re.IGNORECASE)
    sphere_um = int(sphere_match.group(1)) if sphere_match else np.nan

    # Octree parameters: initial, grad threshold, final
    octree_match = re.search(r'_initial(\d+)max([-+]?\d*\.?\d+)final(\d+)', base)
    if octree_match:
        octree_params = {
            "initial_depth": int(octree_match.group(1)),
            "percent_gradThreshold": float(octree_match.group(2)),
            "final_depth": int(octree_match.group(3))
        }
    else:
        octree_params = {"initial_depth": np.nan, "grad_threshold": np.nan, "final_depth": np.nan}

    metadata = {
        "filename": base,
        "case": case,
        "temperature": temperature,
        "sphere_um": sphere_um,
        "octree": octree_params,
    }

    return metadata


# --- worker for one file ---
def process_file(fileIN):
    metadata = parse_filename_metadata(fileIN)

    if metadata is None:
        return None

    print(f"→ Started {os.path.basename(fileIN)}\n", flush=True)
    processedResults = load_h5_to_dict(fileIN)
    key_prefix = os.path.basename(fileIN).split('_through')[0]

    keys = list(processedResults.keys())

    # Dictionary to hold results for each target point
    target_point_results = {}

    # Process each target point
    for tp_idx, target_point in enumerate(target_points_array):
        tp_key = f"target_{tp_idx:04d}"
        
        # --- sequential processing across keys for this target point ---
        results = []
        for k in keys:
            result = process_key_target(k, processedResults[k], target_point, FIELD_AVERAGE_RADIUS)
            results.append(result)

        ids, E_vecs, E_mags, standard_errors, N = zip(*results)
        ids = np.array(ids)
        E_vecs = np.array(E_vecs)
        E_mags = np.array(E_mags)
        standard_errors = np.array(standard_errors)
        num_points = np.array(N)

        target_point_results[tp_key] = {
            "iter": ids,
            "E_vecs": E_vecs,
            "E_mags": E_mags,
            "point_errors": standard_errors,
            "N": num_points,
            "target_point": target_point,
            "radius": FIELD_AVERAGE_RADIUS
        }
        
        if (tp_idx + 1) % 10 == 0:
            print(f"  Processed {tp_idx + 1}/{len(target_points_array)} target points", flush=True)

    return key_prefix, {
        "target_points": target_point_results,
        "metadata": metadata
    }

# --- sequential execution across files ---
all_processed = {}

for fileIN in h5_filenames:
    try:
        result = process_file(fileIN)
        if result is not None:
            key_prefix, data = result
            all_processed[key_prefix] = data
            print(f"✔ Finished {os.path.basename(fileIN)}\n", flush=True)
    except Exception as e:
        print(f"❌ Error in {os.path.basename(fileIN)}: {e}\n", flush=True)

print(f"\n=== Processing Complete ===")
print(f"Total files processed: {len(all_processed)}")
print(f"Target points per file: {len(target_points_array)}")

# --- Access results example ---
# all_processed[key_prefix]["target_points"]["target_0000"]["E_vecs"]
# all_processed[key_prefix]["target_points"]["target_0000"]["target_point"]
# all_processed[key_prefix]["metadata"]

In [ ]:
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 14})

# select one key for multiple target points
KEY_TARGET_MULTI = 'PE_425K_initial8max0.8final11_noDissipation_sphere40um_pos-0.1' #'SW_425K_initial8max0.8final10_RefinedGridDissipation_sphere40um_pos-0.1' # 'SW_425K_initial8max0.8final11_RefinedGridDissipation_sphere40um_pos-0.1'

# Load external literature data
zimmerman_SWdata = pd.read_csv("literature-data/Fig7a-SW.csv")
zimmerman_PEdata = pd.read_csv("literature-data/Fig7a-PE.csv")
zimmerman_PEandSWdata = pd.read_csv("literature-data/Fig7a-PE+SW.csv")

# Simulation Parameters (used for time conversion)
WORLD_XY_AREA_SQ_M = 400 * 300 / (1e6**2)  # World area (m^2)
PE_Ele_FLUX = 4e-6 * 6.241509e18
SW_ION_FLUX = 3e-7 * 6.241509e18
SW_Ele_FLUX = 1.5e-6 * 6.241509e18 # still dont know why the SW ele flux gives a difference conversion factor

# PE Conversion Factor
PARTICLES_PER_ITERATION_PE = 81775
CONVERT_ITERATION_PE_TIME = PARTICLES_PER_ITERATION_PE / WORLD_XY_AREA_SQ_M / PE_Ele_FLUX

# SW Conversion Factor
PARTICLES_PER_ITERATION_SW = 14208
CONVERT_ITERATION_SW_TIME = PARTICLES_PER_ITERATION_SW / WORLD_XY_AREA_SQ_M / SW_ION_FLUX

# SW Conversion Factor
PARTICLES_PER_ITERATION_All = 4452
CONVERT_ITERATION_All_TIME = PARTICLES_PER_ITERATION_All / WORLD_XY_AREA_SQ_M / SW_ION_FLUX

# --- 3. PLOTTING SETUP ---
fig, ax_main = plt.subplots(figsize=(8.01, 4.6))

# --- 4. PLOT LITERATURE DATA ---
ax_main.plot(10**zimmerman_SWdata["x"], zimmerman_SWdata[" y"], '-', color="k", lw=3, alpha=0.3, label=None)
ax_main.plot(10**zimmerman_PEdata["x"], zimmerman_PEdata[" y"], '-', color="k", lw=3, alpha=0.3, label=None)
ax_main.plot(10**zimmerman_PEandSWdata["x"], zimmerman_PEandSWdata[" y"], '--', color="k", lw=3, alpha=0.3)

# --- 5. PLOT SIMULATION DATA FOR EACH TARGET POINT OR FOR ALL CONFIGURATIONS ---

if len(all_processed[list(all_processed.keys())[0]]["target_points"].keys()) == 1:

    print(f"All Factor: {CONVERT_ITERATION_All_TIME*1000:.3f} ms/iteration")
    print(f"PE Factor: {CONVERT_ITERATION_PE_TIME*1000:.3f} ms/iteration")
    print(f"SW Factor: {CONVERT_ITERATION_SW_TIME*1000:.3f} ms/iteration")

    # Color Map Setup
    CMAP_NAME = 'jet'
    n_targets = len(all_processed.keys())
    discrete_cmap = plt.get_cmap(CMAP_NAME, n_targets + 1)
    color_list_rgba = [discrete_cmap(i) for i in np.linspace(0, 1, n_targets + 1)]

    for selectkey, colorIN in zip(all_processed.keys(),color_list_rgba):

        # Get case type for time conversion
        case = all_processed[selectkey]["metadata"]["case"]
        if case == "PE": factor = CONVERT_ITERATION_PE_TIME
        elif case == "SW": factor = CONVERT_ITERATION_SW_TIME
        else: factor = CONVERT_ITERATION_All_TIME

        targetIN =list(all_processed[selectkey]["target_points"].keys())[0]
            
        # Extract data
        x_data = np.array(all_processed[selectkey]["target_points"][targetIN]["iter"]-1) * factor
        y_data = abs(all_processed[selectkey]["target_points"][targetIN]["E_vecs"][:, 0])
        y_err = all_processed[selectkey]["target_points"][targetIN]["point_errors"][:, 0]
        target_point = all_processed[selectkey]["target_points"][targetIN]["target_point"]
        
        # Plot the data line
        ax_main.plot(x_data, y_data, '-', color=colorIN, lw=1.5, 
                    label=f'{selectkey.split("_")[0]} case: {selectkey.split("_")[2]}')
        
        # Optional: Add error region
        # ax_main.fill_between(x_data, y_data - y_err, y_data + y_err, 
        #                      color=colorIN, alpha=0.15, label=None)

    ax_main.legend(loc="lower right", fontsize=8, ncol=1)
else: 

    selectkey = KEY_TARGET_MULTI
    print(f"Plotting multiple target points for key: {selectkey}")

    # Color Map Setup
    CMAP_NAME = 'jet'
    n_targets = len(all_processed[selectkey]["target_points"].keys())
    discrete_cmap = plt.get_cmap(CMAP_NAME, n_targets + 1)
    color_list_rgba = [discrete_cmap(i) for i in np.linspace(0, 1, n_targets + 1)]

    # Get case type for time conversion
    case = all_processed[selectkey]["metadata"]["case"]
    if case == "PE": factor = CONVERT_ITERATION_PE_TIME
    elif case == "SW": factor = CONVERT_ITERATION_SW_TIME
    else: factor = CONVERT_ITERATION_All_TIME

    print(f"{case} Factor: {factor*1000:.3f} ms/iteration")

    # Plot each target point
    for targetIN, colorIN in zip(all_processed[selectkey]["target_points"].keys(), color_list_rgba):
        
        # Extract data
        x_data = np.array(all_processed[selectkey]["target_points"][targetIN]["iter"]-1) * factor
        y_data = abs(all_processed[selectkey]["target_points"][targetIN]["E_vecs"][:, 0])
        y_err = all_processed[selectkey]["target_points"][targetIN]["point_errors"][:, 0]
        target_point = all_processed[selectkey]["target_points"][targetIN]["target_point"]
        
        # Plot the data line
        ax_main.plot(x_data, y_data, '-', color=colorIN, lw=1.5, 
                    label=f'x:{target_point[0]:.3f} z:{target_point[2]:.3f}')
        
        # Optional: Add error region
        # ax_main.fill_between(x_data, y_data - y_err, y_data + y_err, 
        #                      color=colorIN, alpha=0.15, label=None)

    ax_main.legend(loc="upper left", fontsize=8, ncol=2)

# --- 6. FORMAT MAIN PLOT ---
ax_main.set_xlabel("Time [s]")
ax_main.set_ylabel(r"$|E_x|$ (V/m)")
# ax_main.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
# ax_main.set_ylim(0, 2.2e5)
ax_main.set_xlim(1e-1, 6)
#ax_main.set_ylim(0,1e5)
ax_main.grid(True, linestyle=':', alpha=0.5)

# --- 7. SAVE AND SHOW ---
plt.tight_layout()
plt.savefig("figures/field_evolution_target_points.jpeg", bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 12})

# --- 1. CONFIGURATION AND DATA LOADING ---
print("\n--- Starting Data Processing and Plot Generation ---")

# Define Data Keys for Subtraction
KEY_FIT = 'PE_425K_initial8max0.8final11_noDissipation_sphere40um_pos-0.1'
KEY_TARGET = 'PE_425K_initial8max0.8final11_RefinedGridDissipation_sphere40um_pos-0.1'

# NOTE: Since `all_processed` is not defined, the code below assumes it is a loaded dictionary
# and will execute if the rest of your environment is set up.

# Load external literature data
zimmerman_SWdata = pd.read_csv("literature-data/Fig7a-SW.csv")
zimmerman_PEdata = pd.read_csv("literature-data/Fig7a-PE.csv")
zimmerman_PEandSWdata = pd.read_csv("literature-data/Fig7a-PE+SW.csv")

# Simulation Parameters (used for time conversion)
WORLD_XY_AREA_SQ_M = 400 * 300 / (1e6**2)  # World area (m^2)
PE_Ele_FLUX = 4e-6 * 6.241509e18
SW_ION_FLUX = 3e-7 * 6.241509e18
SW_Ele_FLUX = 1.5e-6 * 6.241509e18 # still dont know why the SW ele flux gives a difference conversion factor

# PE Conversion Factor
PARTICLES_PER_ITERATION_PE = 81775
CONVERT_ITERATION_PE_TIME = PARTICLES_PER_ITERATION_PE / WORLD_XY_AREA_SQ_M / PE_Ele_FLUX

# SW Conversion Factor
PARTICLES_PER_ITERATION_SW = 14208
CONVERT_ITERATION_SW_TIME = PARTICLES_PER_ITERATION_SW / WORLD_XY_AREA_SQ_M / SW_ION_FLUX

# SW Conversion Factor
PARTICLES_PER_ITERATION_All = 4452
CONVERT_ITERATION_All_TIME = PARTICLES_PER_ITERATION_All / WORLD_XY_AREA_SQ_M / SW_ION_FLUX

# Color Map Setup
CMAP_NAME = 'Dark2'
discrete_cmap = plt.get_cmap(CMAP_NAME, 8 + 1)
color_list_rgba = [discrete_cmap(i) for i in np.linspace(0, 1, 8 + 1)]

# --- Define specific color assignments ---
# Request: SW (index 1), PE (index 2), All (index 4), Difference (index 2)
CASE_COLOR_MAP = {
    'SW': color_list_rgba[1],
    'PE': color_list_rgba[2],
    'All': color_list_rgba[4], # Assuming 'All' is the case type for combined PE+SW
}
# ---

# --- 2. RAW SUBTRACTION AND DIFFERENCE CALCULATION ---

print("\n--- Performing Raw Subtraction (No Extrapolation) ---")

# 1. Extract Raw Data for KEY_FIT
raw_fit_data = all_processed[KEY_FIT]['target_points']['target_0000']
x_fit_raw = np.array(raw_fit_data["iter"] - 1) * CONVERT_ITERATION_PE_TIME
y_fit_raw = abs(raw_fit_data["E_vecs"][:, 0])
y_fit_errors_raw = raw_fit_data["point_errors"][:, 0]

# 2. Extract Raw Data for KEY_TARGET
raw_target_data = all_processed[KEY_TARGET]['target_points']['target_0000']
x_target_raw = np.array(raw_target_data["iter"] - 1) * CONVERT_ITERATION_PE_TIME
y_target_raw = abs(raw_target_data["E_vecs"][:, 0])
y_target_errors_raw = raw_target_data["point_errors"][:, 0]

# 3. Truncate to the length of the shortest array
len_fit = len(x_fit_raw)
len_target = len(x_target_raw)
min_len = min(len_fit, len_target)

print(f"Truncating arrays to length: {min_len} (Fit len: {len_fit}, Target len: {len_target})")

# Truncated arrays
x_common = x_fit_raw[:min_len]
y_fit_trunc = y_fit_raw[:min_len]
y_target_trunc = y_target_raw[:min_len]
y_fit_errors_trunc = y_fit_errors_raw[:min_len]
y_target_errors_trunc = y_target_errors_raw[:min_len]

# Renaming for subsequent plotting code consistency (using the Fit data for the main plot line)
x_plot = x_common
y_plot_fit = y_fit_trunc # The data used as the basis (formerly y_extrapolated)
y_plot_target = y_target_trunc # The data subtracted (formerly y_target)

# 4. Raw Subtraction and Percent Difference Calculation
y_raw_difference = abs(y_plot_fit - y_plot_target)
method_label = f"Raw Subtraction: |E_{KEY_FIT.split('_')[1]}| - |E_{KEY_TARGET.split('_')[1]}|"

# Define Percent Difference Denominator (KEY_FIT data)
y_denominator = np.where(y_plot_fit == 0, 1e-10, y_plot_fit) 
y_percent_diff = (y_raw_difference / y_denominator) * 100 

# --- Calculate Error Propagation for Difference and Percentage ---
# Error propagation for the difference: sqrt(error_fit^2 + error_target^2)
y_raw_difference_errors = np.sqrt(y_fit_errors_trunc**2 + y_target_errors_trunc**2)

# Error propagation for the percentage
# Using the simpler (more conservative) error formula for the difference term
y_percent_diff_errors = (y_raw_difference_errors / np.abs(y_denominator)) * 100 

# Renaming for plotting sections (3-6)
x_extrapolate = x_plot
y_extrapolated = y_plot_fit 
# ---------------------------------------------------

# --- 3. PLOTTING SETUP (MAIN + SUBPLOT) ---

# Set up figure and grid layout (5:1 height ratio for main plot vs. residual plot)
fig = plt.figure(figsize=(7.01, 3.22))
gs = gridspec.GridSpec(2, 1, hspace=0.1, height_ratios=[5, 1])

# Main Plot (Top)
ax_main = fig.add_subplot(gs[0])
# Subtraction Plot (Bottom), sharing the x-axis
ax_sub = fig.add_subplot(gs[1], sharex=ax_main)

# --- 4. MAIN PLOT GENERATION (ax_main) ---

# Plot reference data (Zimmerman)
ax_main.plot(10**zimmerman_SWdata["x"], zimmerman_SWdata[" y"], '-', color="k", lw=3, alpha=0.3, label=None)
ax_main.plot(10**zimmerman_PEdata["x"], zimmerman_PEdata[" y"], '-', color="k", lw=3, alpha=0.3)
ax_main.plot(10**zimmerman_PEandSWdata["x"], zimmerman_PEandSWdata[" y"], '--', color="k", lw=3, alpha=0.3)

# The old `color_list` is now replaced by the `CASE_COLOR_MAP` logic below.
# Removed old line: color_list = [color_list_rgba[1],color_list_rgba[2],color_list_rgba[5],color_list_rgba[6]]
# Removed old line: i=0

# Plot the KEY_FIT data (Truncated) - Using the requested PE color (index 2)
# NOTE: KEY_FIT is 'PE_425K...' so it should use the PE color.
key_fit_case = KEY_FIT.split('_')[0] # 'PE'
ax_main.plot(x_plot, y_plot_fit, '--', color=CASE_COLOR_MAP[key_fit_case], lw=1.5, 
             label=f"Fit Case ({key_fit_case})") # Changed to '--' to distinguish KEY_FIT/KEY_TARGET

# Reset the counter for the main loop, if needed, but the logic should rely on `case`
i = 0 
for keyIN in all_processed.keys():
    
    # Define plotting variables outside of loop to use them later
    case = all_processed[keyIN]["metadata"]["case"]
    
    if case == "PE": factor = CONVERT_ITERATION_PE_TIME
    elif case == "SW": factor = CONVERT_ITERATION_SW_TIME
    elif case == "All": factor = CONVERT_ITERATION_All_TIME
    plot_color = CASE_COLOR_MAP[case]
    
    # Target and Temperature info (if needed for filtering/labeling, but not for color)
    tempIN = all_processed[keyIN]["metadata"]["temperature"]
    
    x_data = np.array(all_processed[keyIN]['target_points']['target_0000']["iter"] - 1) * factor
    y_data = abs(all_processed[keyIN]['target_points']['target_0000']["E_vecs"][:, 0])
    y_err = all_processed[keyIN]['target_points']['target_0000']["point_errors"][:, 0]
    
    print(keyIN)
    
    # Skip plotting the KEY_FIT line if it was already plotted above, or handle KEY_TARGET
    # For now, plot all data using the new mapping.
    
    # Plot the data line
    ax_main.plot(x_data, y_data, '-', color=plot_color, lw=1.5)
    
    # Use fill_between for the error region (Replaces errorbars)
    ax_main.fill_between(x_data, y_data - y_err, y_data + y_err, 
                        color=plot_color, alpha=0.15, 
                        label=None) # Set label=None to avoid extra legend entry

# Add original uncommented features back to ax_main
#ax_main.axvline(x=65 * CONVERT_ITERATION_PE_TIME, color='gray', linestyle='-.', lw=1, alpha=0.7, label="Vertical Marker")
ax_main.set_ylabel(r"$|E_x|$ (V/m)")
ax_main.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
#ax_main.set_yscale('log') # Use log scale for better visualization of power-law decay
ax_main.set_ylim(0,2.4e5) # Uncommented limits
# ax_main.set_xlim(7.4e-1, 1.25e1) # Uncommented limits

# Clean up main plot
# ax_main.grid(True, linestyle=':', alpha=0.5)
# ax_main.legend(loc='lower left', fontsize=8, ncol=2)
# Remove X-tick labels from the main plot
plt.setp(ax_main.get_xticklabels(), visible=False) 
ax_sub.set_ylabel(r"% Diff")

# --- 5. SUBTRACTION PLOT GENERATION (ax_sub) ---

# PLOT PERCENT DIFFERENCE WITH SHADED ERROR REGION
# Color updated to index 2 (color_list_rgba[2])
ax_sub.plot(x_extrapolate, y_percent_diff, '-', color=color_list_rgba[2], lw=2,
            label=r"Relative Error: $\frac{|E_{Fit}| - |E_{Target}|}{|E_{Fit}|}$")
# Shaded region
ax_sub.fill_between(x_extrapolate, y_percent_diff - y_percent_diff_errors, 
                    y_percent_diff + y_percent_diff_errors, 
                    color=color_list_rgba[2], alpha=0.2, label="Error Region")
ax_sub.set_xlabel("Lunar Equivalent Time [s]")
ax_sub.set_ylabel(r"% Diff")
# ax_sub.ticklabel_format(axis='y', style='sci', scilimits=(0, 0)) # Removed, % difference is typically not sci notation
# ax_sub.grid(True, linestyle=':', alpha=0.6)
# ax_sub.legend(loc='upper right', fontsize=8)
ax_sub.set_xlim(0,6) # Uncommented limit check (if sharing xlim works)
ax_sub.set_ylim(0,10) # Uncommented limit check (if sharing xlim works)
ax_sub.set_yticks([0,4,8])

# --- 6. SAVE AND SHOW ---
plt.savefig("figures/zimmerman_benchmark_summary.jpeg", bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
discrete_cmap

## 3D Representation of the Electric Pressure

In [ ]:
iteration_SW, iteration_PE, iteration_All = 57, 131, 180
print(f"SW equivalent time for iteration#{iteration_SW}: {(iteration_SW-1)*CONVERT_ITERATION_SW_TIME}")
print(f"PE equivalent time for iteration#{iteration_PE}: {(iteration_PE-1)*CONVERT_ITERATION_PE_TIME}")
print(f"PE equivalent time for iteration#{iteration_All}: {(iteration_All-1)*CONVERT_ITERATION_All_TIME}")

In [ ]:
configIN = "onlyphotoemission"
directory = "raw-files/ZimmermanInitial8max0.8final11"
#directory = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/build-dissipationRefinedGrid-initial8max0.8final12"

if iteration_PE <10 :
    filenames = sorted(glob.glob(f"{directory}/*00{iteration_PE}*{configIN}*.txt")) #{iteration}
else:
    filenames = sorted(glob.glob(f"{directory}/*{iteration_PE}*{configIN}*.txt")) #{iteration}
print(filenames)

df_PE  = read_data_format_efficient(filenames,scaling=True)

In [ ]:
configIN = "onlysolarwind"
directory = "raw-files/ZimmermanInitial8max0.8final11"
#directory = "/storage/scratch1/5/avira7/Grain-Charging-Simulation-Data/build-dissipationRefinedGrid-initial8max0.8final12"

if iteration_SW <10 :
    filenames = sorted(glob.glob(f"{directory}/*00{iteration_SW}*{configIN}*.txt")) #{iteration}
else:
    filenames = sorted(glob.glob(f"{directory}/*{iteration_SW}*{configIN}*.txt")) #{iteration}
print(filenames)

df_SW = read_data_format_efficient(filenames,scaling=True)

In [ ]:
df_electrons = pd.read_pickle("processed-fieldmaps/PE_zimmerman_electrons_locations_until44.pkl") #get it flipped when saving the files
df_holes = pd.read_pickle("processed-fieldmaps/PE_zimmerman_protons_locations_until44.pkl")

In [ ]:
df_electrons

In [ ]:
# -----------------------------------------
# Step 0: Load field data and filter
# -----------------------------------------
fieldIN = df_PE[iteration_PE]
 
geometry_center = geometry.centroid
red_point = np.array([-0.1, 0., 0.1 + 0.037]) + geometry_center
 
start_time = time.time()
points = fieldIN["pos"]
vectors = fieldIN["E"]
magnitudes = fieldIN["E_mag"]
 
initial_mask = (magnitudes > 0) & (points[:,1]>=-0.1+geometry_center[1]) & (points[:,1]<=0.1+geometry_center[1])
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]
 
epsilon_0 = 8.854187817e-12
 
# Create field cloud
field_cloud = pv.PolyData(points)
field_cloud["E_x"] = vectors[:, 0]
field_cloud["E_y"] = vectors[:, 1]
field_cloud["E_z"] = vectors[:, 2]
 
print(f"Starting points (filtered): {len(points)}")

# ----------------------------------------------------
# Step 1: Create FULL geometry (before cropping)
# ----------------------------------------------------
start_time_geo = time.time()
 
pv_spheres_full = pv.PolyData(
    geometry.vertices,
    np.hstack([np.full((len(geometry.faces), 1), 3), geometry.faces])
).compute_normals()

print(f"Full geometry has {pv_spheres_full.n_cells} faces")

# ============================================================
# Step 2: Calculate surface charge on FULL geometry
# ============================================================
num_faces_full = pv_spheres_full.n_cells
face_charges_full = np.zeros(num_faces_full)

# Face areas (m^2) for full geometry
face_areas_m2_full = pv_spheres_full.compute_cell_sizes()['Area'] * 1e-6 + 1e-20

q_proton = +1.602e-19
q_electron = -1.602e-19

# Get face centers for full geometry
face_centers_full = pv_spheres_full.cell_centers().points

# Build KDTree for fast nearest neighbor search
tree = cKDTree(face_centers_full)

# --- Particle positions ---
e_pos = np.array(df_electrons["Post_Step_Position_mm"].tolist())
p_pos = np.array(df_protons["Post_Step_Position_mm"].tolist())

# --- Bin electrons to closest face ---
if len(e_pos) > 0:
    _, face_id_e = tree.query(e_pos, k=1)
    unique_faces, counts = np.unique(face_id_e, return_counts=True)
    face_charges_full[unique_faces] += counts * q_electron
    print(f"Binned {len(e_pos)} electrons to {len(unique_faces)} unique faces")

# --- Bin protons to closest face ---
if len(p_pos) > 0:
    _, face_id_p = tree.query(p_pos, k=1)
    unique_faces, counts = np.unique(face_id_p, return_counts=True)
    face_charges_full[unique_faces] += counts * q_proton
    print(f"Binned {len(p_pos)} protons to {len(unique_faces)} unique faces")

# --- Calculate surface charge density on full geometry ---
sigma_per_face_SI_full = face_charges_full / face_areas_m2_full
sigma_per_face_uC_full = sigma_per_face_SI_full * 1e6

# Store charge data on full geometry
pv_spheres_full.cell_data['charge_density_SI'] = sigma_per_face_SI_full
pv_spheres_full.cell_data['charge_density_uC'] = sigma_per_face_uC_full

# ============================================================
# Step 3: Crop geometry to field bounds
# ============================================================
bbox_bounds = field_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)

pv_spheres_cropped = (
    pv_spheres_full
    .clip_box(bbox, invert=False)
    .extract_surface()
    .compute_normals(point_normals=True, cell_normals=True, inplace=False)
)

print(f"Cropped geometry has {pv_spheres_cropped.n_cells} faces")

# ============================================================
# Step 4: Interpolate E-field to cropped face centers
# ============================================================
face_centers_cropped = pv_spheres_cropped.cell_centers().points
 
face_field_cloud = pv.PolyData(face_centers_cropped)
face_field_interp = face_field_cloud.interpolate(
    field_cloud,
    radius=0.002,
    strategy='closest_point',
    sharpness=3.0,
    null_value=0.0
)
 
# Extract face-centered E-fields
E_x_faces = face_field_interp["E_x"]
E_y_faces = face_field_interp["E_y"]
E_z_faces = face_field_interp["E_z"]
E_vec_faces = np.stack([E_x_faces, E_y_faces, E_z_faces], axis=1)
 
# Face normals (cropped geometry)
face_normals = pv_spheres_cropped.cell_normals
nx = face_normals[:, 0]   # x-direction component
 
# ============================================================
# Step 5: Compute Maxwell electric pressure (normal)
# ============================================================
E_dot_n = np.einsum('ij,ij->i', E_vec_faces, face_normals)
E_mag_sq = np.einsum('ij,ij->i', E_vec_faces, E_vec_faces)
 
# Normal pressure (scalar)
P_normal_faces = epsilon_0 * (E_dot_n**2 - 0.5 * E_mag_sq)
 
# ============================================================
# Step 6: Compute X-directed electric pressure
# ============================================================
P_x_faces = P_normal_faces * nx
 
# Save electric pressure to cropped mesh
pv_spheres_cropped.cell_data["electric_pressure_x"] = P_x_faces
pv_spheres_cropped.cell_data["E_x"] = E_x_faces
 
print(f"Computed x-directed electric pressure in {time.time() - start_time_geo:.2f}s")

# ============================================================
# Step 7: Calculate F_x using charge density (Q * E_x)
# ============================================================
# Note: The cropped mesh inherited charge_density from the clipping operation
# So we can use it directly
sigma_cropped = pv_spheres_cropped.cell_data['charge_density_SI']

# Calculate force: F = Q * E_x (not sigma * E_x * nx, that's wrong!)
# The correct formula is F_x = Q_face * E_x
pv_spheres_cropped.cell_data['F_x'] = sigma_cropped * E_x_faces #* nx

In [ ]:
pv_spheres_cropped.cell_data['F_x'] = nx #sigma_cropped * E_x_faces #* nx

In [ ]:
nx

In [ ]:
# ============================================================
# Plotting using Px
# ============================================================
pl = pv.Plotter()
pl.set_background('white')
 
pl.add_mesh(
    pv_spheres_cropped,
    scalars="charge_density_SI",
    cmap="seismic",
    opacity=1,
    show_edges=False,
    clim=[-0.5e-5, 0.5e-5],
    interpolate_before_map=False,
    preference="cell"
)
 
pl.view_xy()
pl.show() #jupyter_backend='static'

In [ ]:


# ============================================================
# Plotting using Px
# ============================================================
pl = pv.Plotter()
pl.set_background('white')
 
pl.add_mesh(
    pv_spheres_cropped,
    scalars="electric_pressure_x",
    cmap="seismic",
    opacity=1,
    show_edges=False,
    clim=[vmin, vmax],
    interpolate_before_map=False,
    preference="cell"
)
 
pl.view_xy()
pl.show(jupyter_backend='static')
 
# ============================================================
# Extract Line Plot Data (y=0 top surface)
# ============================================================
y_tolerance = 0.01
z_min = 0.08
 
x_centers = face_centers[:, 0]
y_centers = face_centers[:, 1]
z_centers = face_centers[:, 2]
 
Px_faces = P_x_faces  # shorthand
 
line_mask = (np.abs(y_centers) < y_tolerance) & (z_centers > z_min)
 
x_line = x_centers[line_mask]
z_line = z_centers[line_mask]
Px_line = Px_faces[line_mask]
 
# Bin + average
x_bin_width = 0.005
x_min, x_max = x_line.min(), x_line.max()
x_bins = np.arange(x_min, x_max + x_bin_width, x_bin_width)
x_bin_centers = (x_bins[:-1] + x_bins[1:]) / 2
 
bin_indices = np.digitize(x_line, x_bins)
 
x_line_avg, z_line_avg, Px_line_avg = [], [], []
 
for i in range(1, len(x_bins)):
    mask = (bin_indices == i)
    if mask.any():
        x_line_avg.append(x_line[mask].mean())
        z_line_avg.append(z_line[mask].mean())
        Px_line_avg.append(Px_line[mask].mean())
 
# Sort
x_line_sorted_PE = np.array(x_line_avg)
z_line_sorted_PE = np.array(z_line_avg)
pressure_line_sorted_PE = np.array(Px_line_avg)
 
sort_idx = np.argsort(x_line_sorted_PE)
x_line_sorted_PE = x_line_sorted_PE[sort_idx]
z_line_sorted_PE = z_line_sorted_PE[sort_idx]
pressure_line_sorted_PE = pressure_line_sorted_PE[sort_idx]
 
print(f"Extracted {len(x_line)} raw points, averaged into {len(x_line_sorted_PE)} bins along y=0 line")

In [ ]:
# -----------------------------------------
# Your existing setup code here...
# -----------------------------------------
fieldIN = df_SW[iteration_SW]
vmin, vmax = (-0.05, 0.05) 
 
geometry_center = geometry.centroid
red_point = np.array([-0.1, 0., 0.1 + 0.037]) + geometry_center
 
# ----------------------------------------------------
# Step 0: Filter data
# ----------------------------------------------------
start_time = time.time()
points = fieldIN["pos"]
vectors = fieldIN["E"]
magnitudes = fieldIN["E_mag"]
 
initial_mask = (magnitudes > 0) & (points[:,1]>=-0.1+geometry_center[1]) & (points[:,1]<=0.1+geometry_center[1])
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]
 
epsilon_0 = 8.854187817e-12
 
# Create field cloud
field_cloud = pv.PolyData(points)
field_cloud["E_x"] = vectors[:, 0]
field_cloud["E_y"] = vectors[:, 1]
field_cloud["E_z"] = vectors[:, 2]
 
print(f"Starting points (filtered): {len(points)}")
 
# ----------------------------------------------------
# Step 1: Geometry Setup
# ----------------------------------------------------
start_time_geo = time.time()
 
pv_spheres = pv.PolyData(
    geometry.vertices,
    np.hstack([np.full((len(geometry.faces), 1), 3), geometry.faces])
).compute_normals()
 
bbox_bounds = field_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = (
    pv_spheres
    .clip_box(bbox, invert=False)
    .extract_surface()
    .compute_normals(point_normals=True, cell_normals=True, inplace=False)
)
 
# ============================================================
# Interpolate field to face centers
# ============================================================
face_centers = pv_spheres_cropped.cell_centers().points
 
face_field_cloud = pv.PolyData(face_centers)
face_field_interp = face_field_cloud.interpolate(
    field_cloud,
    radius=0.002,
    strategy='closest_point',
    sharpness=3.0,
    null_value=0.0
)
 
# Extract face-centered E-fields
E_x_faces = face_field_interp["E_x"]
E_y_faces = face_field_interp["E_y"]
E_z_faces = face_field_interp["E_z"]

E_vec_faces = np.stack([E_x_faces, E_y_faces, E_z_faces], axis=1)
#E_vec_mag_faces = np.sqrt(E_x_faces**2 + E_y_faces **2 + E_z_faces **2)
# Face normals
face_normals = pv_spheres_cropped.cell_normals
nx = face_normals[:, 0]   # <-- x-direction component

# ============================================================
# Compute Maxwell electric pressure (normal)
# ============================================================
E_dot_n = np.einsum('ij,ij->i', E_vec_faces, face_normals)
E_mag_sq = np.einsum('ij,ij->i', E_vec_faces, E_vec_faces)
 
# Normal pressure (scalar)
P_normal_faces = epsilon_0 * (E_dot_n**2 - 0.5 * E_mag_sq)
#P_normal_faces = epsilon_0 * (E_vec_mag_faces**2 ) /2 #- 0.5 * E_mag_sq)
 
# ============================================================
# Compute X-directed electric pressure
# ============================================================
P_x_faces = P_normal_faces * nx   # <-- THIS IS WHAT YOU WANTED
 
# Save to cell data
pv_spheres_cropped.cell_data["electric_pressure_x"] = P_x_faces
 
print(f"Computed x-directed electric pressure in {time.time() - start_time_geo:.2f}s")
 
# ============================================================
# Plotting using Px
# ============================================================
pl = pv.Plotter()
pl.set_background('white')
 
pl.add_mesh(
    pv_spheres_cropped,
    scalars="electric_pressure_x",
    cmap="seismic",
    opacity=1,
    show_edges=False,
    clim=[vmin, vmax],
    interpolate_before_map=False,
    preference="cell"
)
 
pl.view_xy()
pl.show() #jupyter_backend='static'
 

In [ ]:
E_vec_faces.magitude??

In [ ]:
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 14})

cmap = plt.cm.seismic 
#vmin, vmax = (-1, 1)
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

fig, ax = plt.subplots(figsize=(5, 0.1))

# --- 1. Create and Configure the ScalarFormatter ---
formatter = ticker.ScalarFormatter(useMathText=True)

formatter.set_useOffset(False) 
formatter.set_powerlimits((0, 0)) 

# --- 2. Create the Colorbar and apply the Formatter ---
cb = mpl.colorbar.ColorbarBase(
    ax, 
    cmap=cmap, 
    norm=norm, 
    orientation='horizontal', label=r"Electric Pressure, x (Pa)"
)

# Apply the formatter to the colorbar's x-axis
cb.ax.xaxis.set_major_formatter(formatter)

# --- 3. Display the Plot ---
plt.show()

In [ ]:
# Enable LaTeX rendering
mpl.rcParams['text.usetex'] = False
# Set the global font size
mpl.rcParams.update({'font.size': 14})

cmap = plt.cm.seismic 
vmin, vmax = (-1,1) 
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

fig, ax = plt.subplots(figsize=(5, 0.1))

# --- 1. Create and Configure the ScalarFormatter ---
formatter = ticker.ScalarFormatter(useMathText=True)

formatter.set_useOffset(False) 
formatter.set_powerlimits((0, 0)) 

# --- 2. Create the Colorbar and apply the Formatter ---
cb = mpl.colorbar.ColorbarBase(
    ax, 
    cmap=cmap, 
    norm=norm, 
    orientation='horizontal', label=r"Electric Pressure, x (Pa)"
)

# Apply the formatter to the colorbar's x-axis
cb.ax.xaxis.set_major_formatter(formatter)

# --- 3. Display the Plot ---
plt.show()

In [ ]:
df_protons = pd.read_pickle("processed-fieldmaps/SW_zimmerman_electrons_locations_until44.pkl") #get it flipped when saving the files
df_electrons = pd.read_pickle("processed-fieldmaps/SW_zimmerman_protons_locations_until44.pkl")

In [ ]:

def plot_charge_density(electrons, protons, convex_combined, vmin=0, vmax=1e-3):
 
    # --- 1. Initialize Charge and Area Arrays ---
    num_faces = len(convex_combined.faces)
    face_charges = np.zeros(num_faces)
    face_areas_m2 = convex_combined.area_faces * 1e-6
    # Add a small epsilon to prevent division by zero for any zero-area faces
    face_areas_m2 += 1e-20

    q_proton = +1.602e-19
    q_electron = -1.602e-19
 
    # --- 2. Get Particle Positions ---
    # We get positions in mm, as the trimesh object is in mm
    e_pos = np.array(electrons["Post_Step_Position_mm"].tolist())
    p_pos = np.array(protons["Post_Step_Position_mm"].tolist())
 
    # --- 3. Bin Electrons and Protons to Faces to get Q_face ---
    # Bin electrons
    if len(e_pos) > 0:
        # Find the closest face ID for each electron
        _, _, face_id_e = convex_combined.nearest.on_surface(e_pos)
        # Find unique faces and how many electrons hit each one
        unique_faces_e, counts_e = np.unique(face_id_e, return_counts=True)
        # Add the total charge from these electrons to the face_charges array
        # Q_face = N_electrons * q_electron
        face_charges[unique_faces_e] += (counts_e * q_electron)
 
    # Bin protons
    if len(p_pos) > 0:
        # Find the closest face ID for each proton
        _, _, face_id_p = convex_combined.nearest.on_surface(p_pos)
        # Find unique faces and how many protons hit each one
        unique_faces_p, counts_p = np.unique(face_id_p, return_counts=True)
        # Add the total charge from these protons to the face_charges array
        # Q_face = N_protons * q_proton
        face_charges[unique_faces_p] += (counts_p * q_proton)
 
    # --- 4. Calculate Surface Charge Density (sigma) ---
    # sigma = Q_face / A_face (in uC/m^2)
    sigma_per_face = face_charges*1e6 / face_areas_m2
    # # --- 5. Calculate Electric Pressure (P) ---
    # # P = sigma^2 / (2 * epsilon_0) (in Pascals, N/m^2)
    # face_pressures = (sigma_per_face**2) / (2.0 * epsilon_0)
 
    # --- 6. Apply Colormap ---
    colors_rgba = np.zeros((num_faces, 4), dtype=np.uint8)
    # Pressure is always positive (it's squared), so use a sequential colormap
    # like 'hot', 'viridis', or 'plasma' instead of 'seismic'.
    cmap = plt.cm.BrBG #plt.cm.hot
    norm_func = Normalize(vmin=vmin, vmax=vmax)
    colors_rgb = cmap(norm_func(sigma_per_face))[:, :3]
 
    # Assign RGB to all faces
    colors_rgba[:, :3] = (colors_rgb * 255).astype(np.uint8)
    # Set alpha to opaque so faces are visible
    colors_rgba[:, 3] = 255
 
    # Apply colors to mesh
    convex_combined.visual.face_colors = colors_rgba
 
    return convex_combined, sigma_per_face, colors_rgba

In [ ]:
surf, interation0,_ = plot_charge_density(df_electrons, df_protons, geometry, vmin=-5, vmax=5)
# Make sure each triangle has its own unique vertices
surface_edited = surf.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:

# -----------------------------------------
# Your existing setup code here...
# -----------------------------------------
fieldIN = df_SW[iteration_SW]
vmin, vmax = (-0.05, 0.05) 
 
geometry_center = geometry.centroid
 
# ----------------------------------------------------
# Step 0: Filter data
# ----------------------------------------------------
start_time = time.time()
points = fieldIN["pos"]
vectors = fieldIN["E"]
magnitudes = fieldIN["E_mag"]
 
initial_mask = (magnitudes > 0) & (points[:,1]>=-0.1+geometry_center[1]) & (points[:,1]<=0.1+geometry_center[1])
points = points[initial_mask]
vectors = vectors[initial_mask]
magnitudes = magnitudes[initial_mask]
 
epsilon_0 = 8.854187817e-12
 
# Create field cloud
field_cloud = pv.PolyData(points)
field_cloud["E_x"] = vectors[:, 0]
field_cloud["E_y"] = vectors[:, 1]
field_cloud["E_z"] = vectors[:, 2]
 
print(f"Starting points (filtered): {len(points)}")

pv_spheres = pv.PolyData(
    geometry.vertices,
    np.hstack([np.full((len(geometry.faces), 1), 3), geometry.faces])
).compute_normals()
 
bbox_bounds = field_cloud.bounds
bbox = pv.Box(bounds=bbox_bounds)
pv_spheres_cropped = (
    pv_spheres
    .clip_box(bbox, invert=False)
    .extract_surface()
    .compute_normals(point_normals=True, cell_normals=True, inplace=False)
)


# ============================================================
# Interpolate field to face centers
# ============================================================
face_centers = pv_spheres_cropped.cell_centers().points
 
face_field_cloud = pv.PolyData(face_centers)
face_field_interp = face_field_cloud.interpolate(
    field_cloud,
    radius=0.002,
    strategy='closest_point',
    sharpness=3.0,
    null_value=0.0
)
 
# Extract face-centered E-fields
E_x_faces = face_field_interp["E_x"]
E_y_faces = face_field_interp["E_y"]
E_z_faces = face_field_interp["E_z"]

# --- 1. Compute face areas ---
face_areas_m2 = pv_spheres_cropped.compute_cell_sizes()['Area'] * 1e-6  # mm^2 -> m^2
face_areas_m2 += 1e-20  # prevent division by zero
# --- 2. Initialize face charges ---
num_faces = pv_spheres_cropped.n_cells
face_charges = np.zeros(num_faces)
q_proton = +1.602e-19
q_electron = -1.602e-19
# --- 3. Bin electrons to faces ---
e_pos = np.array(df_electrons["Post_Step_Position_mm"].tolist())
if len(e_pos) > 0:
    face_id_e = pv_spheres_cropped.find_closest_cell(e_pos)
    valid_mask = face_id_e >= 0
    face_id_e = face_id_e[valid_mask]
    unique_faces, counts = np.unique(face_id_e, return_counts=True)
    face_charges[unique_faces] += counts * q_electron
# --- 4. Bin protons to faces ---
p_pos = np.array(df_protons["Post_Step_Position_mm"].tolist())
if len(p_pos) > 0:
    face_id_p = pv_spheres_cropped.find_closest_cell(p_pos)
    valid_mask = face_id_p >= 0
    face_id_p = face_id_p[valid_mask]
    unique_faces, counts = np.unique(face_id_p, return_counts=True)
    face_charges[unique_faces] += counts * q_proton
# --- 5. Surface charge density (C/m^2) ---
sigma_per_face = face_charges / face_areas_m2
pv_spheres_cropped.cell_data['charge_density'] = sigma_per_face
# --- 6. Extract X-component of face-centered E-field ---
# E_x_faces = face_field_interp["E_x"]
# --- 7. Compute X-directed force per face (F = Q * E_x) ---
# Q_face = sigma * A_face -> F_x = Q_face * E_x = sigma * A * E_x
F_x_faces = sigma_per_face * E_x_faces
pv_spheres_cropped.cell_data['F_x'] = F_x_faces


In [ ]:
interation

In [ ]:
# ============================================================
# Plotting using Px
# ============================================================
pl = pv.Plotter()
pl.set_background('white')
 
pl.add_mesh(
    pv_spheres_cropped,
    scalars="charge_density",
    cmap="seismic",
    opacity=1,
    show_edges=False,
    #clim=[-0.5, 0.5],
    interpolate_before_map=False,
    preference="cell"
)
 
pl.view_xy()
pl.show() #jupyter_backend='static'
 